In [8]:
import argparse
import os
from kabuki.analyze import gelman_rubin
import pandas
import hddm
import glob
import re
import pickle as pkl

path = '/home/gdholla1/projects/bias/data/hddm_fits'

# SELECT MASK


def get_model(model, hemisphere):
    df = pandas.read_pickle('/home/gdholla1/projects/bias/data/behavior/behavior_and_single_trial_estimates.pandas')

    # Prepare data
    data = df
    data['response'] = data['correct']
    data['rt'] = data['rt'] / 1000.
    data = data[(data.rt > .2) & (data.rt < 1.5)]
    data = data[~data.rt.isnull()]
    data['cue_coding'] = data.cue_validity.map({'valid':1, 'neutral':0, 'invalid':-1})

    def z_link_func(x, data=data):
        return 1 / (1 + np.exp(-(x.values.ravel())))

    if model == 'drift_super':
        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + cue_coding + C(difficulty)*{}_stim + C(difficulty)*{}_stim + C(difficulty)*{}_stim'.format('STh_{}_A'.format(hemisphere),
                                                                                                                 'STh_{}_B'.format(hemisphere),
                                                                                                                 'STh_{}_C'.format(hemisphere),), 
                 'link_func': lambda x: x}
        
        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv'), bias=True, group_only_regressors=False, group_only_nodes=['sv'])

    if model == 'drift_errors_super':
        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': ('v ~ 1 + '
                          'C(difficulty)*{}_stim + {}_stim:correct + '
                          'C(difficulty)*{}_stim + {}_stim:correct + ' 
                          'C(difficulty)*{}_stim + {}_stim:correct').format('STh_{}_A'.format(hemisphere), 'STh_{}_A'.format(hemisphere),
                                                                            'STh_{}_B'.format(hemisphere), 'STh_{}_B'.format(hemisphere),
                                                                            'STh_{}_C'.format(hemisphere), 'STh_{}_C'.format(hemisphere)), 'link_func': lambda x: x}
        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv'), bias=True, group_only_regressors=True, group_only_nodes=['sv'])

    if model == 'start_point_super':
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + cue_coding + C(difficulty)', 'link_func': lambda x: x}
        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv'), bias=True, group_only_regressors=False, group_only_nodes=['sv'])

    if model == 'all_super':
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        #v_reg = {'model': ('v ~ 1 + '
                          #'C(difficulty)*{}_stim + {}_stim:correct + '
                          #'C(difficulty)*{}_stim + {}_stim:correct + ' 
                          #'C(difficulty)*{}_stim + {}_stim:correct').format('STh_{}_A'.format(hemisphere), 'STh_{}_A'.format(hemisphere),
                                                                            #'STh_{}_B'.format(hemisphere), 'STh_{}_B'.format(hemisphere),
                                                                            #'STh_{}_C'.format(hemisphere), 'STh_{}_C'.format(hemisphere)), 'link_func': lambda x: x}        
        v_reg = {'model': ('v ~ 1 + '
                          'C(difficulty)*{}_stim  + '
                          'C(difficulty)*{}_stim + ' 
                          'C(difficulty)*{}_stim').format('STh_{}_A'.format(hemisphere), 'STh_{}_B'.format(hemisphere), 'STh_{}_C'.format(hemisphere), ), 'link_func': lambda x: x}        


    if model == 'all_super_drift_bias_cue':
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': ('v ~ 1 + '
                          'C(difficulty)*{}_stim  +'
                          'C(difficulty)*{}_stim + ' 
                          'C(difficulty)*{}_stim + '
                          'cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue').format('STh_{}_A'.format(hemisphere), 'STh_{}_B'.format(hemisphere), 'STh_{}_C'.format(hemisphere), 'STh_{}_A'.format(hemisphere), 'STh_{}_B'.format(hemisphere), 'STh_{}_C'.format(hemisphere), ), 'link_func': lambda x: x}        

        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv'), bias=True, group_only_regressors=False, group_only_nodes=['sv'])

    if model == 'all_super_drift_bias_stim':
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': ('v ~ 1 + '
                          'C(difficulty)*{}_stim  +'
                          'C(difficulty)*{}_stim + ' 
                          'C(difficulty)*{}_stim + '
                          'cue_coding + cue_coding:{}_stim + cue_coding:{}_stim + cue_coding:{}_stim').format('STh_{}_A'.format(hemisphere), 'STh_{}_B'.format(hemisphere), 'STh_{}_C'.format(hemisphere), 'STh_{}_A'.format(hemisphere), 'STh_{}_B'.format(hemisphere), 'STh_{}_C'.format(hemisphere), ), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv'), bias=True, group_only_regressors=False, group_only_nodes=['sv'])

    if model == 'all_super_drift_bias_stim_sv_sz_p_outlier':
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': ('v ~ 1 + '
                          'C(difficulty)*{}_stim  +'
                          'C(difficulty)*{}_stim + ' 
                          'C(difficulty)*{}_stim + '
                          'cue_coding + cue_coding:{}_stim + cue_coding:{}_stim + cue_coding:{}_stim').format('STh_{}_A'.format(hemisphere), 'STh_{}_B'.format(hemisphere), 'STh_{}_C'.format(hemisphere), 'STh_{}_A'.format(hemisphere), 'STh_{}_B'.format(hemisphere), 'STh_{}_C'.format(hemisphere), ), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz'], p_outlier=0.05)

    if model == 'final_super_model_startpoint':
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty) + cue_coding', 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz', 'st'])
    
    if model == 'final_super_model_drift':
        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + C(difficulty)*{}_stim +  C(difficulty)*{}_stim + cue_coding'.format('STh_{}_A'.format(hemisphere),
                                                                                                                               'STh_{}_B'.format(hemisphere),
                                                                                                                               'STh_{}_C'.format(hemisphere),), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz', 'st'])
        
        
    if model == 'final_super_model_startpoint_group':
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty) + cue_coding', 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=True, group_only_nodes=['sv', 'sz', 'st'])


    if model == 'final_super_model_drift_group':
        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + C(difficulty)*{}_stim +  C(difficulty)*{}_stim + cue_coding'.format('STh_{}_A'.format(hemisphere),
                                                                                                                               'STh_{}_B'.format(hemisphere),
                                                                                                                               'STh_{}_C'.format(hemisphere),), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=True, group_only_nodes=['sv', 'sz', 'st'])

    if model == 'final_super_model_startpoint_zscored':
        
        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'cue_validity'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'cue_validity'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())
            
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty) + cue_coding', 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        
        
        
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz', 'st'])        
        
        
    if model == 'final_super_model_startpoint_group_zscored':
        
        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'cue_validity'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'cue_validity'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())
            
        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue + cue_coding:{}_cue + cue_coding:{}_cue'.format('STh_{}_A'.format(hemisphere),
                                                                                                                  'STh_{}_B'.format(hemisphere),
                                                                                                                  'STh_{}_C'.format(hemisphere),), 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty) + cue_coding', 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        
        
        
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=True, group_only_nodes=['sv', 'sz', 'st'])        
        
    if model == 'final_super_model_drift_zscored':
        
        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())
            
        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + C(difficulty)*{}_stim +  C(difficulty)*{}_stim + cue_coding'.format('STh_{}_A'.format(hemisphere),
                                                                                                                               'STh_{}_B'.format(hemisphere),
                                                                                                                               'STh_{}_C'.format(hemisphere),), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz', 'st'])
        
    if model == 'final_super_model_drift_group_zscored':
        
        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())
            
        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + C(difficulty)*{}_stim +  C(difficulty)*{}_stim + cue_coding'.format('STh_{}_A'.format(hemisphere),
                                                                                                                               'STh_{}_B'.format(hemisphere),
                                                                                                                               'STh_{}_C'.format(hemisphere),), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=True, group_only_nodes=['sv', 'sz', 'st'])


    if model == 'final_super_model_drift_zscored_error_interaction':

        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())

        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + C(difficulty)*{}_stim +  C(difficulty)*{}_stim + cue_coding'.format('STh_{}_A'.format(hemisphere),
                                                                                                                               'STh_{}_B'.format(hemisphere),
                                                                                                                               'STh_{}_C'.format(hemisphere),), 'link_func': lambda x: x}


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz', 'st'])

    if model == 'final_super_model_drift_group_zscored_error_interaction':

        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())

        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + C(difficulty)*{}_stim +  C(difficulty)*{}_stim + cue_coding'.format('STh_{}_A'.format(hemisphere),
                                                                                                                               'STh_{}_B'.format(hemisphere),
                                                                                                                               'STh_{}_C'.format(hemisphere),), 'link_func': lambda x: x}



        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=True, group_only_nodes=['sv', 'sz', 'st'])
        
    if model.startswith('final_super_model_startpoint_group_zscored_'):

        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())

        mask = model[-7:]

        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue'.format(mask), 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty) + cue_coding', 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=True, group_only_nodes=['sv', 'sz', 'st'])        


    if model.startswith('final_super_model_startpoint_zscored_'):

        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())

        mask = model[-7:]

        z_reg = {'model': 'z ~ 0 + cue_coding + cue_coding:{}_cue'.format(mask), 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty) + cue_coding', 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz', 'st'])        

        
        
    if model.startswith('final_super_model_drift_group_zscored_'):

        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())

        mask = model[-7:]

        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + cue_coding'.format(mask), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=True, group_only_nodes=['sv', 'sz', 'st'])        


    if model.startswith('final_super_model_drift_zscored_'):

        for mask in ['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C']:
            data['%s_cue' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_cue' % mask].transform(lambda x: (x - x.mean())/x.std())
            data['%s_stim' % mask] = data.groupby(['subj_idx', 'difficulty', 'response'])['%s_stim' % mask].transform(lambda x: (x - x.mean())/x.std())

        mask = model[-7:]

        z_reg = {'model': 'z ~ 0 + cue_coding', 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + C(difficulty)*{}_stim + cue_coding'.format(mask), 'link_func': lambda x: x}        


        reg_descr = [z_reg, v_reg]
        hddm_model = hddm.HDDMRegressor(data, reg_descr, include=('sv', 'sz', 'st'), bias=True, group_only_regressors=False, group_only_nodes=['sv', 'sz', 'st'])        

        
        
        
    # NOW FIND, LOAD, AND CONCAT TRACES
    reg = re.compile('.*/traces_{model}_{hemisphere}_[0-9]+.pkl'.format(**locals()))
    fns = [fn for fn in glob.glob('/home/gdholla1/projects/bias/data/hddm_fits/traces_{model}_{hemisphere}_*.pkl'.format(**locals())) if reg.match(fn)]

    hddm_model.load_db(fns[0], db='pickle')

    for fn in fns[1:]:
        traces = pkl.load(open(fn))    
        for node in hddm_model.get_stochastics().node:
            node.trace._trace[0] = np.concatenate([traces[str(node)][0], node.trace[:]])     

    return hddm_model
        
        
        
        
    # NOW FIND, LOAD, AND CONCAT TRACES
    reg = re.compile('.*/traces_{model}_{hemisphere}_[0-9]+.pkl'.format(**locals()))
    fns = [fn for fn in glob.glob('/home/gdholla1/projects/bias/data/hddm_fits/traces_{model}_{hemisphere}_*.pkl'.format(**locals())) if reg.match(fn)]

    hddm_model.load_db(fns[0], db='pickle')

    for fn in fns[1:]:
        traces = pkl.load(open(fn))    
        for node in hddm_model.get_stochastics().node:
            node.trace._trace[0] = np.concatenate([traces[str(node)][0], node.trace[:]])     

    return hddm_model


Couldn't import dot_parser, loading of dot files will not be possible.


/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [1]:
import pandas

In [2]:
df = pandas.read_pickle('/home/gdholla1/projects/bias/data/behavior/behavior_and_single_trial_estimates.pandas')

In [6]:
df[['subj_idx', 'cue', 'difficulty', 'correct', 'rt', 'STh_R_A_cue', 'STh_R_B_cue', 'STh_R_C_cue']].to_csv('data_udo.csv')

In [9]:
results = []

for hemisphere in ['L', 'R']:
    hddm_model = get_model('final_super_model_startpoint_group_zscored', hemisphere)
    
    traces = hddm_model.get_traces()
    
    for segment in ['A', 'B', 'C']:
        tmp = pandas.DataFrame({'value':traces['z_cue_coding:STh_{hemisphere}_{segment}_cue'.format(**locals())],
                          'segment':segment,
                          'hemisphere':hemisphere})
        
        results.append(tmp)

Adding these covariates:
['z_cue_coding', 'z_cue_coding:STh_L_A_cue', 'z_cue_coding:STh_L_B_cue', 'z_cue_coding:STh_L_C_cue']
Adding these covariates:
['v_Intercept', 'v_C(difficulty)[T.hard]', 'v_cue_coding']
Adding these covariates:
['z_cue_coding', 'z_cue_coding:STh_R_A_cue', 'z_cue_coding:STh_R_B_cue', 'z_cue_coding:STh_R_C_cue']
Adding these covariates:
['v_Intercept', 'v_C(difficulty)[T.hard]', 'v_cue_coding']


In [10]:
df = pandas.concat(results, ignore_index=True)
df.groupby(['hemisphere', 'segment']).apply(lambda x: (x.value > 0).mean())

hemisphere  segment
L           A          0.819283
            B          0.548700
            C          0.504200
R           A          0.537567
            B          0.258800
            C          0.936183
dtype: float64

In [13]:
traces.to_csv('traces_udo.csv')

In [ ]:
df